In [1]:
// Usual setup: importing Semantic Kernel SDK and SkiaSharp, used to display images inline.

#r "nuget: Microsoft.SemanticKernel, 1.11.1"
#r "nuget: SkiaSharp, 2.88.3"

#!import config/Settings.cs
#!import config/Utils.cs
#!import config/SkiaUtils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.TextToImage;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Installed Packages Microsoft.SemanticKernel, 1.11.1 SkiaSharp, 2.88.3

Loading extensions from `C:\Users\mikokono\.nuget\packages\skiasharp\2.88.3\interactive-extensions\dotnet\SkiaSharp.DotNet.Interactive.dll`

In [ ]:
using Kernel = Microsoft.SemanticKernel.Kernel;

#pragma warning disable SKEXP0001, SKEXP0010

// Load OpenAI credentials from config/settings.json
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile("config/settings.json");

// Configure the two AI features: OpenAI Chat and DALL-E 3 for image generation
var builder = Kernel.CreateBuilder();

if(useAzureOpenAI)
{
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
    builder.AddAzureOpenAITextToImage("dall-e-3", azureEndpoint, apiKey);
}
else
{
    builder.AddOpenAIChatCompletion("gpt-3.5-turbo", apiKey, orgId);
    builder.AddOpenAITextToImage(apiKey, orgId);
}

var kernel = builder.Build();

// Get AI service instance used to generate images
var dallE = kernel.GetRequiredService<ITextToImageService>();

// Get AI service instance used to manage the user chat
var chatGPT = kernel.GetRequiredService<IChatCompletionService>();

In [ ]:
var systemMessage = "You're chatting with a user. Instead of replying directly to the user"
                  + " provide a description of a cartoonish image that expresses what you want to say."
                  + " The user won't see your message, they will see only the image."
                  + " Describe the image with details in one sentence.";

var chat = new ChatHistory(systemMessage);

In [ ]:
#pragma warning disable SKEXP0001

while (true)
{
    // 1. Ask the user for a message. The user enters a message.  Add the user message into the Chat History object.
    var userMessage = await InteractiveKernel.GetInputAsync("Your message");
    Console.WriteLine($"User: {userMessage}");
    chat.AddUserMessage(userMessage);

    // 2. Send the chat object to AI asking to generate a response. Add the bot message into the Chat History object.
    var assistantReply = await chatGPT.GetChatMessageContentAsync(chat, new OpenAIPromptExecutionSettings());
    chat.AddAssistantMessage(assistantReply.Content);

    // 3. Show the reply as an image
    Console.WriteLine($"\nBot:");
    var imageUrl = await dallE.GenerateImageAsync(assistantReply.Content, 1024, 1024);
    await SkiaUtils.ShowImage(imageUrl, 1024, 1024);
    Console.WriteLine($"[{assistantReply}]\n");
}